# Advanced Deep Learning Examples

**Created:** 2025-12-04 14:21:45 UTC

This notebook collects advanced, practical deep-learning examples with clear explanations, full training loops, and tips for reproducible experiments. Each section is a self-contained example you can run locally. Requirements (commonly): `torch`, `torchvision`, `transformers`, `datasets`, `accelerate`, `tqdm`, `scikit-learn`.


## 1) CNN Image Classification — Full PyTorch Training Loop

Goal: Train a ResNet18 on CIFAR-10 (or a small custom dataset). This section shows data loading, augmentation, model, training loop with checkpointing, mixed precision, and evaluation.

In [ ]:
# CNN Image Classification - PyTorch (runnable)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets, models
from tqdm import tqdm
import os

# Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = './data/cifar10'
batch_size = 128
num_epochs = 10
lr = 1e-3
checkpoint_dir = './checkpoints/cnn_cifar10'
os.makedirs(checkpoint_dir, exist_ok=True)

# Data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.247,0.243,0.261))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.247,0.243,0.261))
])

train_ds = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
test_ds = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

# Model (pretrained backbone, small head)
model = models.resnet18(pretrained=False, num_classes=10)
model = model.to(device)

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

def train_one_epoch(model, loader, optimizer, criterion, device, scaler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(loader, desc='Train', leave=False)
    for images, targets in pbar:
        images = images.to(device); targets = targets.to(device)
        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == targets).sum().item()
        total += images.size(0)
        pbar.set_postfix(loss=running_loss/total, acc=100*correct/total)
    return running_loss/total, 100*correct/total

def evaluate(model, loader, device):
    model.eval()
    correct = 0; total = 0; loss_sum = 0.0
    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device); targets = targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss_sum += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += images.size(0)
    return loss_sum/total, 100*correct/total

best_acc = 0.0
for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device, scaler)
    val_loss, val_acc = evaluate(model, test_loader, device)
    scheduler.step()
    print(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%')
    # checkpoint
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'epoch': epoch, 'model_state': model.state_dict(), 'optimizer_state': optimizer.state_dict()}, os.path.join(checkpoint_dir, 'best.pth'))
print('Training complete. Best val acc:', best_acc)


## 2) Transfer Learning — Fine-tuning a Pretrained CNN

Goal: Use a pretrained ImageNet model, replace the head and fine-tune for a new dataset. Tips: frozen backbone, staged unfreezing, differential learning rates.

In [ ]:
# Transfer learning sketch (PyTorch)
from torchvision import models
import torch.nn as nn

model = models.resnet50(pretrained=True)
# Replace final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)  # e.g., 5 classes on new dataset
# Freeze backbone
for name, param in model.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False
# Train head first, then unfreeze some layers and fine-tune with smaller LR


## 3) Transformers for NLP — Fine-tuning BERT with Hugging Face

Goal: Fine-tune a pretrained transformer (BERT) for sequence classification using `transformers` + `datasets`. Uses Trainer API for brevity and reproducibility.

In [ ]:
# Transformers fine-tuning (using Hugging Face Trainer)
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

dataset = load_dataset('glue', 'mrpc')  # example
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
def preprocess(batch):
    return tokenizer(batch['sentence1'], batch['sentence2'], truncation=True, padding='max_length', max_length=128)
dataset = dataset.map(preprocess, batched=True)
dataset = dataset.rename_column('label', 'labels')
dataset.set_format(type='torch', columns=['input_ids','attention_mask','labels'])

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
args = TrainingArguments(output_dir='./results', evaluation_strategy='epoch', per_device_train_batch_size=16, num_train_epochs=3, save_total_limit=2)
trainer = Trainer(model=model, args=args, train_dataset=dataset['train'], eval_dataset=dataset['validation'])
trainer.train()


## 4) Generative Adversarial Networks (GANs) — Basic DCGAN Sketch

Goal: Understand generator/discriminator training alternating steps and stabilization tips (batchnorm, label smoothing, spectral norm).

In [ ]:
# DCGAN simplified sketch (PyTorch)
# This is a condensed sketch; for full training follow dcgan tutorials.
import torch.nn as nn
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super().__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            # more layers...
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # more layers...
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x).view(-1,1).squeeze(1)
# Training alternates: update D on real+fake, then update G to fool D.


## 5) Deep Reinforcement Learning — DQN / PPO Practical Notes

Goal: Use existing libraries (stable-baselines3, rl-baselines3-zoo) for reliable implementations. Included is a short sketch and tips for reproducibility and hyperparameters.

In [ ]:
# Deep RL recommendation (sketch) - install stable-baselines3
# pip install stable-baselines3[extra]
# Example usage:
# from stable_baselines3 import DQN
# import gym
# env = gym.make('CartPole-v1')
# model = DQN('MlpPolicy', env, learning_rate=1e-3, buffer_size=50000, exploration_fraction=0.1, verbose=1)
# model.learn(total_timesteps=100000)
# model.save('dqn_cartpole')


## Additional tips

- Use deterministic seeds and document software/hardware.
- Log with TensorBoard or Weights & Biases.
- Profile training for bottlenecks (data loading, augmentation).
- Use mixed precision and gradient accumulation for large-batch training on limited GPUs.
